# Magic Squares Implementation
resource blog: https://medium.com/qiskit/this-proof-demonstrates-a-quantum-advantage-even-for-noisy-quantum-computers-b44a738801ad

In [1]:
from qiskit import *
from qiskit.visualization import plot_histogram
import numpy as np

In [2]:
#The following function sets up the initial entanglement between Alice and Bob
def share_bell_state(qc,a,b,c,d): 
    qc.h(a)
    qc.h(b)
    qc.cx(a,c)
    qc.cx(b,d)
    
#The following functions represent the U(gamma) and V(gamma) controlled Cliffords.
def U(qc,gamma,a,b):
    if gamma==1:
        qc.h(a)
        qc.i(b)
    
    elif gamma==2:
        qc.h(a)
        qc.i(b)
        qc.swap(a,b)
    
    elif gamma==3:
        qc.h(a)
        qc.i(b)
        qc.cx(a,b)
    
def V(qc,gamma,a,b):
    if gamma==1:
        qc.h(a)
        qc.h(b)
    
    if gamma==2:
        qc.swap(a,b)
    
    elif gamma==3:
        qc.h(a)
        qc.h(b)
        qc.cz(a,b)
        qc.z(a)
        qc.z(b)

In [4]:
def run_circuit(alpha, beta):
    #Create the quantum register and the classical register to store our final bit values
    aliceQR = QuantumRegister(2)
    x1CR = ClassicalRegister(1)
    x2CR = ClassicalRegister(1)
    bobQR = QuantumRegister(2)
    y1CR = ClassicalRegister(1)
    y2CR = ClassicalRegister(1)

    #Create the circuit
    magicsquare_circuit = QuantumCircuit(aliceQR,bobQR,x1CR,x2CR,y1CR,y2CR)

    #Generate the Bell state on the circuit
    share_bell_state(magicsquare_circuit,0,1,2,3)

    magicsquare_circuit.barrier()

    #Draw the rest of the circuit based on Alice and Bob's selection   
    if 4>alpha>0 and 4>beta>0:
        U(magicsquare_circuit,alpha,0,1) 
        V(magicsquare_circuit,beta,2,3)
        magicsquare_circuit.measure(0,0)
        magicsquare_circuit.measure(1,1)
        magicsquare_circuit.measure(2,2)
        magicsquare_circuit.measure(3,3) 

    else:
        print("please enter values of 1, 2, or 3")
        
    return magicsquare_circuit
    

## Additions to complete code for playing an entire magic square game

In [5]:
def simulate(qc):
    shots = 8192

    backend_sim = Aer.get_backend('qasm_simulator')
    job_sim = execute(qc, backend_sim, shots=shots)
    result_sim = job_sim.result()
    counts = result_sim.get_counts(qc)
#     plot_histogram(counts)

    #0000 --> dcba
    #0010 --> a=0 b=1 c=0 d=0
    #0101 --> a1 b0 c1 d0
    #0111 --> a1 b1 c1 d0
    
    return counts

In [7]:
qc = run_circuit(1, 3)
counts = simulate(qc)
counts

{'0 0 0 0': 1044,
 '0 0 1 0': 1002,
 '0 1 0 1': 1015,
 '0 1 1 1': 1007,
 '1 0 0 0': 994,
 '1 0 1 0': 1053,
 '1 1 0 1': 1057,
 '1 1 1 1': 1020}

In [8]:
def score(counts, alpha, beta):
    print(f'Alpha = {alpha}, Beta = {beta}')
    score = 0
    total_options = len(counts)
    for key, value in zip(counts.keys(),counts.values()):
        
        alice0 = int(key[-1])
        alice1 = int(key[-3])
        alice2 = (alice0 + alice1 + 1) % 2
        
        bob0 = int(key[2])
        bob1 = int(key[0])
        bob2 = (bob0 + bob1) % 2
        
        alice_col = [alice0, alice1, alice2]
        bob_row = [bob0, bob1, bob2]
        
        conflict = False
        if alice_col[beta - 1] == bob_row[alpha - 1]:
            score += 1
        else:
            conflict = True
        
        print(f'alice col: {alice0}{alice1}{alice2}')
        print(f'bob row: {bob0}{bob1}{bob2}')
        
        string_list = ['x','x','x','\n','x','x','x','\n','x','x','x']
        cols = np.array([[0,4,8], [1,5,9], [2,6,10]])
        rows = np.array([[0,1,2], [4,5,6], [8,9,10]])
        for s in range(len(string_list)):
            if s in cols[alpha-1]:
                string_list[s] = str(alice_col[np.where(cols[alpha-1] == s)[0][0]])
            if s in rows[beta-1]:
                string_list[s] = str(bob_row[np.where(rows[beta-1] == s)[0][0]])
        
            if not s in cols[alpha - 1] and not s in rows[beta-1] and not string_list[s] == '\n':
                string_list[s] = ' '
            
            if conflict:
                conflict_at = cols[alpha-1][beta-1]
                string_list[conflict_at] = 'x'
                
        string = ''.join(string_list)
        print(string)
        print()
            
        
    return score, total_options

def score_all():
    total_score = 0
    total_options = 0
    for alpha in range(1,4):
        for beta in range(1,4):
            qc = run_circuit(alpha, beta)
            counts = simulate(qc)
            s, os = score(counts, alpha, beta)
            total_score += s
            total_options += os
    print(f'total_score: {total_score}/{total_options}')
    return total_score
            

In [9]:
# alpha = 3
# beta = 3
# qc = run_circuit(alpha, beta)
# counts = simulate(qc)
# s = score(counts, alpha, beta)
# s

## We find this implementation does not win the game 100% of the time.

In [10]:
score_all()

Alpha = 1, Beta = 1
alice col: 001
bob row: 000
000
0  
1  

alice col: 010
bob row: 000
000
1  
0  

alice col: 100
bob row: 101
101
0  
0  

alice col: 111
bob row: 101
101
1  
1  

alice col: 001
bob row: 011
011
0  
1  

alice col: 010
bob row: 011
011
1  
0  

alice col: 100
bob row: 110
110
0  
0  

alice col: 111
bob row: 110
110
1  
1  

Alpha = 1, Beta = 2
alice col: 001
bob row: 000
0  
000
1  

alice col: 100
bob row: 000
1  
000
0  

alice col: 010
bob row: 101
0  
101
0  

alice col: 111
bob row: 101
1  
101
1  

alice col: 001
bob row: 011
0  
011
1  

alice col: 100
bob row: 011
1  
011
0  

alice col: 010
bob row: 110
0  
110
0  

alice col: 111
bob row: 110
1  
110
1  

Alpha = 1, Beta = 3
alice col: 001
bob row: 000
0  
0  
x00

alice col: 010
bob row: 000
0  
1  
000

alice col: 100
bob row: 101
1  
0  
x01

alice col: 111
bob row: 101
1  
1  
101

alice col: 001
bob row: 011
0  
0  
x11

alice col: 010
bob row: 011
0  
1  
011

alice col: 100
bob row: 110
1  
0  
x1

44